In [1]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.5 MB/s eta 0:00:00


In [2]:
from getpass import getpass
import os

os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")


Enter your Groq API key: ··········


In [3]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [4]:
import json

def classify_complaint(text):
    prompt = f"""
You are a complaint classification system.

Classify the complaint into exactly one of:
- food_spoiled
- food_tampered
- late_delivery
- wrong_item

Return JSON only:
{{
  "category": "",
  "confidence": 0.0
}}

Complaint:
{text}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a strict JSON-only classification engine."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    raw_output = response.choices[0].message.content

    try:
        return json.loads(raw_output)
    except:
        return {"error": "Invalid JSON", "raw": raw_output}


In [5]:
result = classify_complaint("The food quality was not good")
print(result)
print(type(result))


{'category': 'food_spoiled', 'confidence': 0.8}
<class 'dict'>


In [6]:
import json

policies = [
  {
    "id": "food_spoiled",
    "title": "Spoiled Food Policy",
    "content": "If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support."
  },
  {
    "id": "food_tampered",
    "title": "Tampered Food Policy",
    "content": "If food appears tampered, apologize immediately. Offer full refund without delay. Escalate to safety team. Mark delivery partner for investigation."
  },
  {
    "id": "late_delivery",
    "title": "Late Delivery Policy",
    "content": "If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund."
  },
  {
    "id": "wrong_item",
    "title": "Wrong Item Policy",
    "content": "If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days."
  },
  {
        "id": "allergy_issue",
        "title": "Allergen Safety Policy",
        "content": "If a customer reports an allergy issue or allergic reaction, apologize immediately. This is a safety priority. If the customer received an item containing an allergen they requested to exclude, offer an immediate full refund. If the customer reports a medical reaction to an allergen, escalate to the emergency safety team and human support instantly."
    },
    {
        "id": "driver_misconduct",
        "title": "Driver Conduct Policy",
        "content": "If a customer reports driver misconduct, a rude driver, or an unprofessional delivery person, apologize sincerely. If the driver was aggressive, unprofessional, or violated safety rules, escalate the complaint to the logistics management team. If the driver failed to follow specific delivery instructions or was rude, provide a 20 percent discount voucher for the next order."
    },
    {
        "id": "payment_billing",
        "title": "Payment and Billing Policy",
        "content": "If a customer has a payment query, billing error, or was overcharged, verify the order total. If there is a double payment or incorrect billing amount, initiate a refund to the original payment method. For general payment inquiries regarding promo codes, discounts, or credit card errors, provide the billing FAQ link. Escalate complex billing disputes to the finance department."
    },
    {
        "id": "missing_entire_order",
        "title": "Non-Delivery Policy",
        "content": "If an entire order is missing, or the customer reports the order was never delivered despite being marked as completed, apologize. Check the driver's GPS coordinates at the time of delivery. If the order cannot be located or was delivered to the wrong address, offer a full refund or a free redelivery. Escalate to the fraud prevention team if the same customer reports a missing order more than twice."
    },
  {
        "id": "packaging_defect",
        "title": "Packaging Quality Policy",
        "content": "If a customer reports a packaging defect, such as a broken container, leaked sauce, or crushed box that makes the food difficult to eat, apologize. If the packaging failure resulted in food being spilled or inedible, offer a full refund. If the damage is cosmetic only, offer a 10 percent discount."
    },
    {
        "id": "item_quality_texture",
        "title": "Food Texture and Quality Policy",
        "content": "If a customer complains about food quality issues like soggy fries, rubbery steak, or stale bread, apologize. These are quality concerns. Offer a 50 percent refund or a replacement. If the food is reported as 'inedible' due to texture, offer a full refund."
    },
    {
        "id": "technical_app_error",
        "title": "App and Technical Support Policy",
        "content": "If a customer reports a technical error, app crash, or GPS tracking failure, apologize. Verify if the error prevented the order from being placed or tracked. If a technical glitch resulted in a failed delivery, offer a full refund. Escalate persistent app bugs to the engineering team."
    },
    {
        "id": "temperature_issue",
        "title": "Food Temperature Policy",
        "content": "If a customer reports that hot food arrived cold or cold food arrived warm, apologize. If the temperature issue is due to a delivery delay (over 30 mins late), offer a full refund. If the delivery was on time but the temperature is poor, offer a 30 percent discount."
    },
    {
        "id": "dietary_preference_violation",
        "title": "Dietary Preference Policy",
        "content": "If a customer receives a meal that violates a non-allergy dietary preference (e.g., meat in a vegetarian dish, or non-halal meat), apologize sincerely. Offer a replacement or full refund. This policy applies to religious, ethical, or lifestyle dietary choices."
    },
    {
        "id": "missing_utensils_condiments",
        "title": "Missing Add-ons Policy",
        "content": "If a customer reports missing utensils, straws, or small condiments (ketchup, napkins), apologize. Since these are low-value items, offer a small credit ($2-$5) to the customer account. Do not offer full refunds for missing napkins or straws."
    },
    {
        "id": "subscription_billing",
        "title": "Premium Subscription Policy",
        "content": "If a customer has a query about a 'Pro' or 'Gold' subscription, or was charged a monthly fee unexpectedly, explain the cancellation process. If the customer was charged after a trial ended, offer a one-time courtesy refund of the subscription fee and escalate to the accounts team."
    },
    {
        "id": "promotion_issue",
        "title": "Promotion and Voucher Policy",
        "content": "If a voucher code is not working or a 'Buy One Get One' (BOGO) deal was not honored by the restaurant, apologize. Manually apply the discount as a credit to the account if the customer provides a screenshot of the valid promotion."
    },
    {
        "id": "restaurant_service_complaint",
        "title": "Restaurant Performance Policy",
        "content": "If a customer complains about the restaurant's food preparation (e.g., portion size too small, food looks nothing like the photo), apologize. Note the feedback for the restaurant partner. Offer a 20 percent discount if the portion size is significantly smaller than advertised."
    },
    {
        "id": "environmental_policy",
        "title": "Sustainability and Waste Policy",
        "content": "If a customer complains about excessive plastic usage or receiving plastic cutlery despite opting out, apologize. Log the feedback for our sustainability team. This does not qualify for a refund but helps us improve our eco-friendly goals."
    }
]

with open("policies.json", "w") as f:
    json.dump(policies, f, indent=2)


In [54]:
policy_retrieval_texts = [
    "spoiled food, rotten, bad smell, expired, mold, sour, stale, curdled",
    "tampered food, opened seal, broken seal, missing seal, bite marks, foreign object, hair, plastic, metal",
    "late delivery, delayed order, arrived late, waiting long time, over 45 minutes late, 90 minutes late",
    "wrong item, wrong meal, incorrect order, missing item, missing drink, missing sides, different order",
    "allergy issue, allergic reaction, allergen included, peanuts, nuts, shellfish, gluten allergy, dairy allergy",
    "driver misconduct, rude driver, aggressive driver, unsafe driving, harassment, did not follow instructions",
    "payment billing, charged twice, double charge, overcharged, billing error, refund missing, payment failed but charged",
    "missing entire order, not delivered, marked delivered but not received, wrong address delivery, stolen food",
    "packaging defect, broken container, leaked, spilled, crushed box, cracked lid, food spilled",
    "texture issue, soggy fries, rubbery, stale bread, undercooked, overcooked, inedible texture",
    "technical app error, app crash, tracking failure, GPS issue, checkout bug, login error, internal server error",
    "temperature issue, arrived cold, arrived warm, melted, lukewarm, hot food cold, cold food warm",
    "dietary preference violation, vegetarian got meat, vegan got eggs, halal violation, non-halal meat, religious diet",
    "missing utensils, missing spoon, missing fork, missing straw, missing ketchup, missing napkins, missing condiments",
    "subscription billing, pro membership, gold membership, charged subscription, trial charged, cancel subscription refund",
    "promotion issue, voucher not working, promo code invalid, bogo not honored, discount not applied, coupon refused",
    "restaurant service complaint, portion too small, misleading photos, food not like picture, poor preparation",
    "environmental complaint, too much plastic, wasteful packaging, sustainability issue, plastic cutlery despite opting out"
]


In [55]:
assert len(policy_retrieval_texts) == len(policies), (len(policy_retrieval_texts), len(policies))


In [56]:
with open("policies.json", "r") as f:
    policies = json.load(f)


In [57]:
def retrieve_policy(category):
    for policy in policies:
        if policy["id"] == category:
            return policy["content"]
    return "No policy found."


In [58]:
#Test1
result = classify_complaint("My order arrived 1 hour late")
policy_text = retrieve_policy(result["category"])

print("Classification:", result)
print("Policy:", policy_text)


Classification: {'category': 'late_delivery', 'confidence': 1.0}
Policy: If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund.


In [59]:
import json

def generate_response(complaint, category, policy_text):
    prompt = f"""
You are a professional customer support assistant.

You must strictly follow the company policy provided below.

Company Policy:
{policy_text}

Customer Complaint:
{complaint}

Instructions:
- Follow policy strictly.
- Do not invent compensation beyond policy.
- Return JSON only:

{{
  "response_text": "",
  "action_taken": "",
  "escalate": false
}}
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a strict JSON-only customer support AI."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    raw_output = response.choices[0].message.content

    try:
        return json.loads(raw_output)
    except:
        return {"error": "Invalid JSON", "raw": raw_output}


In [60]:
def process_complaint(complaint):
    classification = classify_complaint(complaint)

    if classification.get("confidence", 0) < 0.6:
        return {
            "response_text": "Your complaint requires human review.",
            "action_taken": "Escalated to human support",
            "escalate": True
        }

    policy_text = retrieve_policy(classification["category"])

    response = generate_response(
        complaint,
        classification["category"],
        policy_text
    )

    return response


In [61]:
#Test2
complaint = "My order arrived 1 hour late"

result = classify_complaint(complaint)
policy_text = retrieve_policy(result["category"])

final_response = generate_response(
    complaint,
    result["category"],
    policy_text
)

print(final_response)


{'response_text': 'We apologize for the delay in your order. Since it arrived 1 hour late, which is more than 45 minutes beyond the estimated time, we are offering a 30% refund.', 'action_taken': 'Refund of 30% will be processed', 'escalate': False}


In [62]:
#Test3
print(process_complaint("Service was terrible"))


{'response_text': 'Your complaint requires human review.', 'action_taken': 'Escalated to human support', 'escalate': True}


In [63]:
!pip install sentence-transformers faiss-cpu


In [64]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer("all-mpnet-base-v2")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [65]:
import json

with open("policies.json", "r") as f:
    policies = json.load(f)

policy_texts = [p["title"] + " " + p["content"] for p in policies]


In [66]:
policy_embeddings = embedding_model.encode(
    policy_retrieval_texts,
    normalize_embeddings=True
)

dimension = policy_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(np.array(policy_embeddings))


In [99]:
def retrieve_policy_rag(query, top_k=2):
    query_embedding = embedding_model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(np.array(query_embedding), top_k)

    results = []
    for rank, idx in enumerate(indices[0]):
        results.append({
            "policy_id": policies[idx]["id"],
            "content": policies[idx]["content"],
            "score": float(distances[0][rank])
        })

    return results


In [100]:
#Test 2.4
print(retrieve_policy_rag("I was charged twice for my order", top_k=3))
print(retrieve_policy_rag("My order says delivered but I got nothing", top_k=3))
print(retrieve_policy_rag("The driver was rude and aggressive", top_k=3))
print(retrieve_policy_rag("My fries are soggy and stale", top_k=3))


[{'policy_id': 'payment_billing', 'content': 'If a customer has a payment query, billing error, or was overcharged, verify the order total. If there is a double payment or incorrect billing amount, initiate a refund to the original payment method. For general payment inquiries regarding promo codes, discounts, or credit card errors, provide the billing FAQ link. Escalate complex billing disputes to the finance department.', 'score': 0.5897160768508911}, {'policy_id': 'late_delivery', 'content': 'If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund.', 'score': 0.43415117263793945}, {'policy_id': 'missing_entire_order', 'content': "If an entire order is missing, or the customer reports the order was never delivered despite being marked as completed, apologize. Check the driver's GPS coordinates at the time of delivery. If the order cannot be located or was delivered to the wrong address, offer a full refund

In [101]:
#Test 2.1
print(retrieve_policy_rag("My food smelled bad and looked spoiled"))
print(retrieve_policy_rag("I received the wrong meal"))
print(retrieve_policy_rag("Delivery was very late"))
print(retrieve_policy_rag("What are your working hours?"))


[{'policy_id': 'food_spoiled', 'content': 'If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support.', 'score': 0.6156234741210938}, {'policy_id': 'item_quality_texture', 'content': "If a customer complains about food quality issues like soggy fries, rubbery steak, or stale bread, apologize. These are quality concerns. Offer a 50 percent refund or a replacement. If the food is reported as 'inedible' due to texture, offer a full refund.", 'score': 0.5192832946777344}]
[{'policy_id': 'wrong_item', 'content': 'If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days.', 'score': 0.6672794818878174}, {'policy_id': 'restaurant_service_complaint', 'content': "If a customer complains 

In [102]:
SIMILARITY_THRESHOLD = 0.35   # was 0.50 (too strict)
MARGIN_THRESHOLD = 0.05       # if top1 and top2 are too close → ambiguous → escalate

def decision_layer(user_text):
    results = retrieve_policy_rag(user_text, top_k=2)
    top1 = results[0]
    top2 = results[1] if len(results) > 1 else {"score": 0.0}

    # Escalate if confidence is actually low
    if top1["score"] < SIMILARITY_THRESHOLD:
        return {
            "escalate": True,
            "reason": "Low similarity score",
            "score": top1["score"]
        }

    # Escalate if it's ambiguous (two policies almost equally likely)
    if (top1["score"] - top2["score"]) < MARGIN_THRESHOLD:
        return {
            "escalate": True,
            "reason": "Ambiguous policy match",
            "score": top1["score"],
            "top1": top1["policy_id"],
            "top2": results[1]["policy_id"]
        }

    return {
        "escalate": False,
        "policy": top1
    }


In [103]:
#Test 2.2
print(decision_layer("My food smelled bad"))
print(decision_layer("What are your working hours?"))


{'escalate': False, 'policy': {'policy_id': 'food_spoiled', 'content': 'If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support.', 'score': 0.5730106830596924}}
{'escalate': True, 'reason': 'Low similarity score', 'score': 0.22255848348140717}


In [104]:
def generate_response(user_text, policy_content):

    prompt = f"""
You are an automated customer support agent.

You MUST strictly follow the policy below.
Do NOT invent rules or compensation outside this policy.
If the complaint requires escalation according to the policy, set escalate to true.

Policy:
{policy_content}

Customer Complaint:
{user_text}

Return JSON only:
{{
  "response_text": "",
  "action_taken": "",
  "escalate": false
}}
"""

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # or current Groq model
        messages=[
            {"role": "system", "content": "You output strict JSON only."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [105]:
#Test 2.3
print(retrieve_policy_rag("My food smelled rotten"))
print(retrieve_policy_rag("Delivery came 2 hours late"))
print(retrieve_policy_rag("I got the wrong meal"))
print(retrieve_policy_rag("Do you sell groceries?"))

[{'policy_id': 'food_spoiled', 'content': 'If a customer reports spoiled food, apologize sincerely. Ask for image proof. Offer a full refund. If the same customer reports spoiled food more than twice in 30 days, escalate to human support.', 'score': 0.598553478717804}, {'policy_id': 'item_quality_texture', 'content': "If a customer complains about food quality issues like soggy fries, rubbery steak, or stale bread, apologize. These are quality concerns. Offer a 50 percent refund or a replacement. If the food is reported as 'inedible' due to texture, offer a full refund.", 'score': 0.44709551334381104}]
[{'policy_id': 'late_delivery', 'content': 'If delivery is delayed more than 45 minutes beyond estimated time, offer 30 percent refund. If more than 90 minutes, offer full refund.', 'score': 0.6979739665985107}, {'policy_id': 'missing_entire_order', 'content': "If an entire order is missing, or the customer reports the order was never delivered despite being marked as completed, apologiz

In [106]:
print(retrieve_policy_rag("I received the wrong meal"))


[{'policy_id': 'wrong_item', 'content': 'If a customer receives the wrong item, wrong meal, incorrect order, or missing item, apologize sincerely. Offer replacement or full refund. Ask whether they prefer refund or redelivery. Escalate if repeated more than twice in 30 days.', 'score': 0.6672794818878174}, {'policy_id': 'restaurant_service_complaint', 'content': "If a customer complains about the restaurant's food preparation (e.g., portion size too small, food looks nothing like the photo), apologize. Note the feedback for the restaurant partner. Offer a 20 percent discount if the portion size is significantly smaller than advertised.", 'score': 0.5536714792251587}]


In [107]:
import datetime
interaction_logs = []

def handle_complaint(user_text):

    decision = decision_layer(user_text)

    log_entry = {
        "timestamp": str(datetime.datetime.now(datetime.UTC)),
        "user_text": user_text,
        "similarity_score": None,
        "policy_id": None,
        "escalated": None,
        "response": None
    }

    if decision["escalate"]:
        log_entry["similarity_score"] = decision.get("score")
        log_entry["escalated"] = True
        log_entry["response"] = "Escalated to human support"

        interaction_logs.append(log_entry)

        return {
            "response_text": "Your complaint requires human review.",
            "action_taken": "Escalated to human support",
            "escalate": True
        }

    policy = decision["policy"]
    log_entry["similarity_score"] = policy["score"]
    log_entry["policy_id"] = policy["policy_id"]
    log_entry["escalated"] = False

    response = generate_response(user_text, policy["content"])

    parsed_response = json.loads(response)
    log_entry["response"] = parsed_response


    interaction_logs.append(log_entry)

    return response


In [108]:
import json

def save_logs():
    with open("interaction_logs.json", "w") as f:
        json.dump(interaction_logs, f, indent=2)


In [109]:
print(handle_complaint("My food smelled rotten"))
print(handle_complaint("Delivery was 2 hours late"))
print(handle_complaint("I received the wrong dish"))
print(handle_complaint("Do you sell groceries?"))


{
  "response_text": "We apologize sincerely for the inconvenience. Can you please provide us with an image of the spoiled food as proof?",
  "action_taken": "Offered full refund",
  "escalate": false
}
{
  "response_text": "We apologize for the delay in your delivery. Since your delivery was more than 90 minutes late, we are offering a full refund.",
  "action_taken": "Full refund offered",
  "escalate": false
}
{'response_text': 'Your complaint requires human review.', 'action_taken': 'Escalated to human support', 'escalate': True}
{'response_text': 'Your complaint requires human review.', 'action_taken': 'Escalated to human support', 'escalate': True}


In [110]:
handle_complaint("My food smelled rotten")
print(interaction_logs)


[{'timestamp': '2026-02-08 17:56:21.737256+00:00', 'user_text': 'My food smelled rotten', 'similarity_score': 0.598553478717804, 'policy_id': 'food_spoiled', 'escalated': False, 'response': {'response_text': 'We apologize sincerely for the inconvenience. Can you please provide us with an image of the spoiled food as proof?', 'action_taken': 'Offered full refund', 'escalate': False}}, {'timestamp': '2026-02-08 17:56:22.075343+00:00', 'user_text': 'Delivery was 2 hours late', 'similarity_score': 0.6964002847671509, 'policy_id': 'late_delivery', 'escalated': False, 'response': {'response_text': 'We apologize for the delay in your delivery. Since your delivery was more than 90 minutes late, we are offering a full refund.', 'action_taken': 'Full refund offered', 'escalate': False}}, {'timestamp': '2026-02-08 17:56:22.399968+00:00', 'user_text': 'I received the wrong dish', 'similarity_score': 0.4431217908859253, 'policy_id': None, 'escalated': True, 'response': 'Escalated to human support'}

In [111]:
save_logs()


Evaluation

In [112]:
test_cases = [
    {"text": "My food smelled rotten", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "The food was tampered", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "Delivery was 2 hours late", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "I received the wrong dish", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Do you sell groceries?", "expected_policy": None, "should_escalate": True},
    {"text": "My pizza arrived cold and soggy.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "I got the wrong burger instead of my ordered chicken sandwich.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The delivery was 2 hours late and I missed my meeting.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The salad had a weird smell and tasted spoiled.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "My order was tampered with, someone opened the package.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I ordered fries but got onion rings.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The food was cold and late.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "My meal arrived 90 minutes late.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The delivery bag was damaged and food was spilled.", "expected_policy": "packaging_defect", "should_escalate": False},
    {"text": "I got the wrong item and it was spoiled.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Pizza arrived burnt.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The order never arrived.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "My burger had a hair in it.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I received the wrong drink.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Food was spoiled and delivery was late.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "The sushi was cold when it arrived.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "My dessert was missing from the order.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Order came 45 minutes late.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "The packaging was tampered with.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I got a sandwich instead of a wrap.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The pasta smelled bad.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "The delivery person was rude and late.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "The sauce was missing and the order was wrong.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The food was tampered with and had foreign objects.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I received the wrong pizza toppings.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The chicken smelled rotten.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "My order was 2 hours late and half of it was missing.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "Someone opened my meal before delivery.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The drink I ordered was not delivered.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Food was cold and looked spoiled.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "I got the wrong dessert and it was melted.", "expected_policy": "temperature_issue", "should_escalate": True},
    {"text": "Delivery was late and driver did not follow instructions.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "My package was tampered with, plastic seal broken.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I received a completely different order.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The fries were soggy.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The order came late and food was cold.", "expected_policy": "temperature_issue", "should_escalate": True},
    {"text": "My sandwich was missing an ingredient.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Food packaging was opened before delivery.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I got the wrong side dish.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The pizza was spoiled when I opened it.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "Delivery arrived an hour late.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "The order seemed tampered with.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I received a wrong combo meal.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "My steak was overcooked and smelled bad.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "Food was delayed by 90 minutes.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The package was broken and the food spilled.", "expected_policy": "packaging_defect", "should_escalate": True},
    {"text": "I got a completely different dessert.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "My salad was rotten.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "Delivery came 3 hours late.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "Food was tampered with, seal broken.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I received the wrong item twice in a week.", "expected_policy": "wrong_item", "should_escalate": True},
    {"text": "Can I change my order for tomorrow?", "expected_policy": None, "should_escalate": True},
    {"text": "The milk in my latte is curdled and chunky.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "I found a piece of blue plastic wrap inside my burrito.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "I ordered a steak medium-rare but it's raw and cold.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "Tracker says delivered but my porch is empty.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "There are finger marks in the frosting of my cupcake.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "This chicken has a weird chemical smell to it.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Missing my side order of mac and cheese.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The driver took a massive detour, food is now ice cold.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "The staples on the bag were all ripped out when it arrived.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "My prawns are slimy and smell very fishy.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Sent me a Diet Pepsi instead of the regular Coke I ordered.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The delivery is 40 minutes past the maximum arrival window.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "The safety seal on the drink lid was already peeled off.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The avocado in this bowl is completely black and mushy.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "Ordered a family meal but only received one burger.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The driver is stuck in the same spot on the map for 30 mins.", "expected_policy": "technical_app_error", "should_escalate": False},
    {"text": "Someone definitely ate some of my fries, the box is half empty.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The bread on this sandwich is covered in green mold.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Received a pepperoni pizza instead of the veggie one.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "My lunch arrived after my break ended, over an hour late.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The box was crushed and there's a footprint on it.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The eggs in my breakfast wrap were green and slimy.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Requested no onions due to allergy but the burger is full of them.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "Driver went to the wrong street and the food is now frozen.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "The bag was soaked with something that smells like cigarettes.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The sushi rice is hard as a rock and smells fermented.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "I ordered a large shake but got a small cup.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Delivery guy was 50 minutes late and very aggressive.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "The tamper sticker was cut with a knife.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The meat in this taco tastes sour and old.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Missing my drink and the extra dipping sauces.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The courier is lost and won't answer any calls.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "There's a band-aid inside the takeout container.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The fruit salad is fizzy and clearly fermenting.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "I got a hot latte when I specifically ordered an iced one.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Waiting 90 minutes for a simple sandwich is unacceptable.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The drink carrier was empty but my receipt shows 3 sodas.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Found a piece of metal wire in my noodles.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The pork chops are gray and smell like sulfur.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "I received someone else's order meant for 'Sarah'.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "My food has been 'Out for Delivery' for 2 hours now.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "The pizza box was delivered open and half the cheese is gone.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The soup has a weird white film growing on top.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "I ordered a vegan wrap but there is real chicken in here.", "expected_policy": "dietary_preference_violation", "should_escalate": True},
    {"text": "The driver left the bag at a random house down the street.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "The safety tape was peeled back and stuck back down poorly.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The fish in this poke bowl is mushy and smells putrid.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Requested gluten-free crust but received wheat crust.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "App says delivered but the driver is still miles away.", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "I found a long black hair baked into the pizza crust.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The yogurt is 4 days past the expiration date.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "They forgot the fries that come with the combo.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "My order is 75 minutes late and I want a refund.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "The bag was ripped open and my wings were missing.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The beef in the stir-fry is literally raw.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Ordered a latte but received a cup of hot water.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The driver didn't follow the 'No Contact' instruction.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "There's a lipstick mark on the rim of my coffee cup.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The pasta sauce is bubbling and tastes like vinegar.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "Received a chocolate donut instead of the glazed one.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "It’s been 3 hours and support won't answer me.", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "The container looks like someone took a bite out of the burger.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "This milk smells like it's been left in the sun.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Missing my dessert and my drink.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The driver left the food in the rain on the sidewalk.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "The bag was open and my drink was half empty.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The chicken wings are still frozen in the center.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "Ordered a 12 pack of nuggets but only got 6.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The rider is going in the completely opposite direction.", "expected_policy": "technical_app_error", "should_escalate": False},
    {"text": "Found a twist-tie in my mashed potatoes.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The steak has a grayish color and smells rotten.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "Received a Sprite instead of a 7-Up.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Driver was an hour late and was very rude.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "The pizza was delivered vertically, all toppings are ruined.", "expected_policy": "packaging_defect", "should_escalate": True},
    {"text": "The salad greens are slimy and brown.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "I got a wrap instead of the bowl I paid for.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Delivery is 2 hours late and cold.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "I found a bug in my salad.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The cheese on my nachos is hard and smells like plastic.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "Missing the extra side of rice I paid for.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The app crashed and now I can't track my late order.", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "The drink lid has a hole poked through it that wasn't for a straw.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The turkey meat is slimy and iridescent.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "I received a vanilla shake instead of chocolate.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The driver didn't follow delivery instructions at all.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "The bag was wide open and napkins were scattered everywhere.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The bread in this sandwich is stale and rock hard.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "I got a coffee but it has no sugar or cream like requested.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The delivery window was 12:00, it is now 1:30.", "expected_policy": "late_delivery", "should_escalate": False},
    {"text": "There are bite marks on the side of the sandwich.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "The cream in the pastry tastes extremely sour.", "expected_policy": "food_spoiled", "should_escalate": False},
    {"text": "I ordered a burger combo but only the burger arrived.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "The rider stopped at another house for 40 minutes.", "expected_policy": "technical_app_error", "should_escalate": False},
    {"text": "The pizza box was crushed and the sauce is everywhere.", "expected_policy": "packaging_defect", "should_escalate": True},
    {"text": "The chicken tastes like it has been sitting out all day.", "expected_policy": "food_spoiled", "should_escalate": True},
    {"text": "I received the wrong flavor of wings.", "expected_policy": "wrong_item", "should_escalate": False},
    {"text": "Delivery took so long the restaurant is now closed.", "expected_policy": "late_delivery", "should_escalate": True},
    {"text": "I found a staple inside my burrito wrap.", "expected_policy": "food_tampered", "should_escalate": True},
    {"text": "What are your delivery hours?", "expected_policy": None, "should_escalate": True},
    {"text": "Do you offer any discounts for first-time users?", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "I need help with a refund from yesterday.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "I specifically asked for no peanuts due to a severe allergy, but this pad thai is covered in them. I feel a reaction starting.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The driver was incredibly rude to me at the door and used foul language. I feel very uncomfortable.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "I was charged twice for my pizza order. My bank statement shows two identical transactions.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "The app says my food was delivered 20 minutes ago but my porch is totally empty. Where is my food?", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I received a burger with cheese even though I indicated a dairy allergy in the notes.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The delivery person left my food in the middle of the driveway and yelled at me when I asked why.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "I tried to use a 50% off promo code but the app charged me the full amount anyway.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "My order never arrived. I've been waiting for three hours and the driver is not answering.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "This salad has walnuts in it. I am allergic to tree nuts and could have died if I didn't see them.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The driver was driving recklessly and almost hit my mailbox before handing me the food.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "My credit card was charged $50 but the receipt in the bag says $35. Why the overcharge?", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "The tracker shows the order as 'Delivered' but the photo is of a completely different house.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I have a gluten allergy and the restaurant sent me regular wheat bread by mistake.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The delivery guy refused to come to my door and told me to 'come get it yourself' in a very mean tone.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "I need to change my payment method for the order I just placed.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "I'm standing outside and there is no sign of the driver or my food. It’s been an hour since it was 'delivered'.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "There is shellfish in this dish and I have a deadly seafood allergy. Please help.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The driver was unprofessional and was wearing a dirty shirt while eating some of my fries.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "My refund from last week hasn't shown up in my bank account yet. When will I get my money back?", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "The driver stole my food. I watched him pull up on the app, wait a second, and then drive away.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I requested a soy-free meal but this clearly contains soy sauce. I am having an allergic response.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The courier was very rude and made inappropriate comments about my house.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "I have an unauthorized charge on my account from this app that I didn't make.", "expected_policy": "payment_billing", "should_escalate": True},
    {"text": "The order is missing entirely. The restaurant says the driver picked it up, but it's not here.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I am allergic to eggs and this mayo was on my sandwich despite my 'No Mayo' request.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The delivery person was extremely impatient and kept ringing the bell non-stop.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "Why was I charged a 'service fee' that wasn't shown at checkout?", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "The app says my order is complete but I never received anything.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "This food contains sesame seeds. I am highly allergic and the menu didn't mention them.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The driver threw my food at the door and just walked away without a word.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "I am trying to add a new credit card but the payment screen keeps freezing.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "My order is gone. I think it was delivered to the wrong building again.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I ordered the nut-free cake but I can smell almonds. This is dangerous for me.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The delivery driver was very unprofessional and didn't follow the 'Leave at door' note.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "My total was $25 but I was charged $35. I want a refund for the difference.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "I've been waiting for my order for 2 hours and now it just says 'Cancelled'. Where is my food?", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I accidentally ate a bite of this and it has mushrooms. I am allergic to mushrooms!", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The courier was rude when I asked why they were late.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "The app says my payment failed but the money was still taken from my account.", "expected_policy": "payment_billing", "should_escalate": True},
    {"text": "The driver marked the order as delivered but he is still at the restaurant on the map.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "Is there dairy in your pizza dough? I have a severe allergy.", "expected_policy": "allergy_issue", "should_escalate": False},
    {"text": "The driver left the food in front of my neighbor's door, not mine.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "Can I get a receipt for my payment sent to my email?", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "I've been looking everywhere but my order is nowhere to be found. It is missing.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I specifically ordered a gluten-free meal and this has a 'contains wheat' sticker on it.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The delivery person was shouting on the phone and was very unprofessional during drop off.", "expected_policy": "driver_misconduct", "should_escalate": False},
    {"text": "I want to remove my saved payment information from the system.", "expected_policy": "payment_billing", "should_escalate": False},
    {"text": "The driver called me and said he ate my food because he was hungry. I am not joking.", "expected_policy": "missing_entire_order", "should_escalate": True},
    {"text": "I am having an allergic reaction to the honey in this tea. I asked for no honey.", "expected_policy": "allergy_issue", "should_escalate": True},
    {"text": "The driver was late and acted like it was my fault. Extremely rude behavior.", "expected_policy": "driver_misconduct", "should_escalate": True},
    {"text": "The plastic lid on my curry was cracked and half the sauce is in the bottom of the bag.", "expected_policy": "packaging_defect", "should_escalate": False},
    {"text": "My fries are so soggy they feel like wet cardboard. I can't even pick them up.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The app crashed right after I paid, and I didn't get a confirmation email. Did my order go through?", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "My milkshake is completely melted and warm. It looks like milk.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "I am a strict vegetarian for religious reasons, but there are pieces of bacon in my salad.", "expected_policy": "dietary_preference_violation", "should_escalate": True},
    {"text": "I ordered a salad and soup but I didn't get any spoons or forks. How am I supposed to eat this at work?", "expected_policy": "missing_utensils_condiments", "should_escalate": False},
    {"text": "I was charged $9.99 for a subscription I never signed up for. Please refund me.", "expected_policy": "subscription_billing", "should_escalate": True},
    {"text": "The BOGO deal for the tacos didn't work. I only got one taco but I was charged for the deal.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "The portion of pasta I received is tiny. It looks like a side dish, not a $20 main course.", "expected_policy": "restaurant_service_complaint", "should_escalate": False},
    {"text": "I checked the 'no plastic' box but my bag is full of plastic straws and cutlery. This is so wasteful.", "expected_policy": "environmental_policy", "should_escalate": False},
    {"text": "The box was totally crushed and the pizza is just a ball of dough now.", "expected_policy": "packaging_defect", "should_escalate": False},
    {"text": "This steak is like rubber. I’ve been chewing one piece for five minutes.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The GPS on the app is stuck at the restaurant but the driver just called saying he is outside.", "expected_policy": "technical_app_error", "should_escalate": False},
    {"text": "My pizza arrived ice cold. The box feels like it was in a fridge.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "I ordered a Halal chicken bowl but the receipt says 'Regular Pork'. This is a huge problem.", "expected_policy": "dietary_preference_violation", "should_escalate": True},
    {"text": "You forgot the ketchup packets and the napkins I specifically asked for.", "expected_policy": "missing_utensils_condiments", "should_escalate": False},
    {"text": "I cancelled my Pro membership last month but I see another charge today.", "expected_policy": "subscription_billing", "should_escalate": True},
    {"text": "The promo code 'SAVE50' says it's invalid even though the banner says it's active today.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "The food looks nothing like the pictures in the app. It's very misleading.", "expected_policy": "restaurant_service_complaint", "should_escalate": False},
    {"text": "Why do you guys use so much styrofoam? It's 2026, get better packaging.", "expected_policy": "environmental_policy", "should_escalate": False},
    {"text": "My drink spilled all over the burger because the cup holder was flimsy.", "expected_policy": "packaging_defect", "should_escalate": False},
    {"text": "The bread on my sub is so stale it's literally crumbling into dust.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "I can't log into my account. It says 'Internal Server Error' every time.", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "The sushi is warm. This is dangerous, it should be cold!", "expected_policy": "temperature_issue", "should_escalate": True},
    {"text": "I ordered a vegan burger and got a real beef patty. I haven't eaten meat in 10 years.", "expected_policy": "dietary_preference_violation", "should_escalate": True},
    {"text": "No straw for my iced coffee. I can't drink this while driving.", "expected_policy": "missing_utensils_condiments", "should_escalate": False},
    {"text": "I want a refund for the 'Gold' fee. I didn't use any of the benefits this month.", "expected_policy": "subscription_billing", "should_escalate": False},
    {"text": "The restaurant refused to honor the 'free drink' coupon I had.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "I paid extra for 'Extra Meat' but there's barely any chicken in this bowl.", "expected_policy": "restaurant_service_complaint", "should_escalate": False},
    {"text": "The delivery bag was tied with ten plastic zip ties. That's way too much plastic.", "expected_policy": "environmental_policy", "should_escalate": False},
    {"text": "The paper bag ripped from the bottom and my food fell onto the street.", "expected_policy": "packaging_defect", "should_escalate": False},
    {"text": "This calamari is so overcooked it's like eating a rubber band.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The checkout button is greyed out even though I filled in all the info.", "expected_policy": "technical_app_error", "should_escalate": False},
    {"text": "My soup is lukewarm and the bread is cold. This was supposed to be a hot meal.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "This dish has eggs in it. I am a vegan for ethical reasons and this is upsetting.", "expected_policy": "dietary_preference_violation", "should_escalate": False},
    {"text": "I need a fork! My hotel doesn't have any and the driver didn't bring them.", "expected_policy": "missing_utensils_condiments", "should_escalate": False},
    {"text": "Why was I billed for a yearly subscription when I selected monthly?", "expected_policy": "subscription_billing", "should_escalate": True},
    {"text": "The 'Free Delivery' promo didn't apply and I was charged $5.99 for shipping.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "I ordered a 'Jumbo' wings plate but they are tiny, like popcorn chicken.", "expected_policy": "restaurant_service_complaint", "should_escalate": False},
    {"text": "Stop sending me so many printed flyers in the bag, it's a waste of paper.", "expected_policy": "environmental_policy", "should_escalate": False},
    {"text": "The container melted because the food was too hot, now there is plastic in my rice.", "expected_policy": "packaging_defect", "should_escalate": True},
    {"text": "The rice is undercooked and crunchy. I can't eat this.", "expected_policy": "item_quality_texture", "should_escalate": False},
    {"text": "The app keeps saying I'm in a different city, so I can't order from local places.", "expected_policy": "technical_app_error", "should_escalate": True},
    {"text": "The ice cream arrived as liquid. It took the driver 40 minutes in a hot car.", "expected_policy": "temperature_issue", "should_escalate": False},
    {"text": "I ordered a pepperoni pizza with 'No Pork' and they used real pork pepperoni.", "expected_policy": "dietary_preference_violation", "should_escalate": True},
    {"text": "Where are the chopsticks? I ordered Chinese food for 5 people and no chopsticks.", "expected_policy": "missing_utensils_condiments", "should_escalate": False},
    {"text": "I was charged for a trial that was supposed to be free for 30 days.", "expected_policy": "subscription_billing", "should_escalate": True},
    {"text": "I have a screenshot of the $10 off deal but it didn't apply to my order.", "expected_policy": "promotion_issue", "should_escalate": False},
    {"text": "The burger patty is half the size of the bun. This is a ripoff.", "expected_policy": "restaurant_service_complaint", "should_escalate": False},
    {"text": "Please use paper bags instead of these thick plastic ones next time.", "expected_policy": "environmental_policy", "should_escalate": False}
]

In [113]:
multi_policy_test_cases = multi_policy_test_cases = [
    {"text": "My food smelled rotten", "expected_policies": ["food_spoiled"], "should_escalate": False},
    {"text": "The food was tampered", "expected_policies": ["food_tampered"], "should_escalate": True},
    {"text": "Delivery was 2 hours late", "expected_policies": ["late_delivery"], "should_escalate": True},
    {"text": "I received the wrong dish", "expected_policies": ["wrong_item"], "should_escalate": False},
    {"text": "Do you sell groceries?", "expected_policies": [], "should_escalate": True},
    {"text": "My pizza arrived cold and soggy.", "expected_policies": ["temperature_issue", "item_quality_texture"], "should_escalate": False},
    {"text": "The delivery was 2 hours late and I missed my meeting.", "expected_policies": ["late_delivery"], "should_escalate": True},
    {"text": "The salad had a weird smell and tasted spoiled.", "expected_policies": ["food_spoiled"], "should_escalate": False},
    {"text": "My order was tampered with, someone opened the package.", "expected_policies": ["food_tampered"], "should_escalate": True},
    {"text": "I got the wrong burger instead of my ordered chicken sandwich.", "expected_policies": ["wrong_item"], "should_escalate": False},
    {"text": "The food was cold and late.", "expected_policies": ["temperature_issue", "late_delivery"], "should_escalate": False},
    {"text": "I got the wrong item and it was spoiled.", "expected_policies": ["wrong_item", "food_spoiled"], "should_escalate": True},
    {"text": "My burger had a hair in it.", "expected_policies": ["food_tampered"], "should_escalate": True},
    {"text": "Food was spoiled and delivery was late.", "expected_policies": ["food_spoiled", "late_delivery"], "should_escalate": True},
    {"text": "I specifically asked for no peanuts due to a severe allergy, but this pad thai is covered in them. I feel a reaction starting.", "expected_policies": ["allergy_issue"], "should_escalate": True},
    {"text": "The driver was incredibly rude to me at the door and used foul language. I feel very uncomfortable.", "expected_policies": ["driver_misconduct"], "should_escalate": True},
    {"text": "I was charged twice for my pizza order. My bank statement shows two identical transactions.", "expected_policies": ["payment_billing"], "should_escalate": False},
    {"text": "My order never arrived. I've been waiting for three hours and the driver is not answering.", "expected_policies": ["missing_entire_order"], "should_escalate": True},
    {"text": "I ordered a vegan wrap but there is real chicken in here.", "expected_policies": ["dietary_preference_violation"], "should_escalate": True},
    {"text": "The driver threw my food at the door and just walked away without a word.", "expected_policies": ["driver_misconduct"], "should_escalate": True},
    {"text": "The delivery bag was tied with ten plastic zip ties. That's way too much plastic.", "expected_policies": ["environmental_policy"], "should_escalate": False},
    {"text": "My fries are so soggy they feel like wet cardboard. I can't even pick them up.", "expected_policies": ["item_quality_texture"], "should_escalate": False},
    {"text": "The app crashed right after I paid, and I didn't get a confirmation email. Did my order go through?", "expected_policies": ["technical_app_error", "payment_billing"], "should_escalate": True},
    {"text": "My milkshake is completely melted and warm. It looks like milk.", "expected_policies": ["temperature_issue"], "should_escalate": False},
    {"text": "Ordered a vegan burger and got a real beef patty. I haven't eaten meat in 10 years.", "expected_policies": ["dietary_preference_violation"], "should_escalate": True},
    {"text": "Missing my dessert and my drink.", "expected_policies": ["wrong_item"], "should_escalate": False},
    {"text": "I ordered a large shake but got a small cup.", "expected_policies": ["wrong_item"], "should_escalate": False},
    {"text": "Delivery took so long the restaurant is now closed.", "expected_policies": ["late_delivery"], "should_escalate": True},
    {"text": "The steak has a grayish color and smells rotten.", "expected_policies": ["food_spoiled"], "should_escalate": True},
    {"text": "I accidentally ate a bite of this and it has mushrooms. I am allergic to mushrooms!", "expected_policies": ["allergy_issue"], "should_escalate": True},
    {"text": "The delivery driver was very unprofessional and didn't follow the 'Leave at door' note.", "expected_policies": ["driver_misconduct"], "should_escalate": False},
    {"text": "The checkout button is greyed out even though I filled in all the info.", "expected_policies": ["technical_app_error"], "should_escalate": False},
]
def test_complaint_classification(classifier):
    for case in multi_policy_test_cases:
        predicted_policies = classifier.predict(case["text"])
        assert set(predicted_policies) == set(case["expected_policies"]), f"Failed for: {case['text']}"
        # Optional: test escalation logic
        assert classifier.should_escalate(case["text"]) == case["should_escalate"], f"Escalation mismatch for: {case['text']}"


In [92]:
def evaluate_system(test_cases):
    interaction_logs.clear()
    correct_retrieval = 0
    correct_escalation = 0
    false_escalations = 0

    total = len(test_cases)

    for case in test_cases:
        result = handle_complaint(case["text"])

        predicted_policy = interaction_logs[-1]["policy_id"]
        predicted_escalation = interaction_logs[-1]["escalated"]

        # Retrieval accuracy
        if predicted_policy == case["expected_policy"]:
            correct_retrieval += 1

        # Escalation accuracy
        if predicted_escalation == case["should_escalate"]:
            correct_escalation += 1

        # False escalation
        if predicted_escalation and not case["should_escalate"]:
            false_escalations += 1

    print("Retrieval Accuracy:", correct_retrieval / total)
    print("Escalation Accuracy:", correct_escalation / total)
    print("False Escalation Rate:", false_escalations / total)


In [93]:
def evaluate_multi_policy_system(test_cases):
    correct_retrieval = 0
    correct_escalation = 0
    false_escalations = 0
    total = len(test_cases)

    for case in test_cases:
        _ = handle_complaint(case["text"])
        last_log = interaction_logs[-1]

        predicted_policy = last_log["policy_id"]          # single policy predicted by your system
        predicted_escalation = last_log["escalated"]

        expected_policies = case.get("expected_policies", [])
        expected_escalate = case["should_escalate"]

        # Retrieval for multi-policy: count as correct if predicted policy is in expected list
        if (predicted_policy in expected_policies) or (predicted_policy is None and expected_policies == []):
            correct_retrieval += 1

        # Escalation accuracy
        if predicted_escalation == expected_escalate:
            correct_escalation += 1

        # False escalation
        if predicted_escalation and not expected_escalate:
            false_escalations += 1

    print("Multi Retrieval Accuracy:", correct_retrieval / total)
    print("Multi Escalation Accuracy:", correct_escalation / total)
    print("Multi False Escalation Rate:", false_escalations / total)


In [90]:
evaluate_system(test_cases)
evaluate_multi_policy_system(multi_policy_test_cases)

Retrieval Accuracy: 0.2248062015503876
Escalation Accuracy: 0.5155038759689923
False Escalation Rate: 0.35271317829457366
Multi Retrieval Accuracy: 0.34375
Multi Escalation Accuracy: 0.6875
Multi False Escalation Rate: 0.1875


In [114]:
def analyze_threshold(test_cases):
    interaction_logs.clear()

    esc_scores = []
    ok_scores = []

    for case in test_cases:
        handle_complaint(case["text"])
        last = interaction_logs[-1]
        s = last["similarity_score"]

        if last["escalated"]:
            esc_scores.append(s)
        else:
            ok_scores.append(s)

    print("Handled count:", len(ok_scores), "Escalated count:", len(esc_scores))
    print("Handled scores: min/avg/max =", min(ok_scores), sum(ok_scores)/len(ok_scores), max(ok_scores))
    print("Escalated scores: min/avg/max =", min(esc_scores), sum(esc_scores)/len(esc_scores), max(esc_scores))


In [98]:
analyze_threshold(test_cases)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kgw2d44ze6erffjmqax1d459` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99970, Requested 172. Please try again in 2m2.688s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [115]:
def evaluate_system_v2(test_cases):
    interaction_logs.clear()

    correct_retrieval = 0
    total_retrieval = 0   # only count non-escalated cases

    correct_escalation = 0
    false_escalations = 0

    total = len(test_cases)

    for case in test_cases:
        handle_complaint(case["text"])
        last_log = interaction_logs[-1]

        predicted_policy = last_log["policy_id"]
        predicted_escalation = last_log["escalated"]

        # escalation metrics (always valid)
        if predicted_escalation == case["should_escalate"]:
            correct_escalation += 1
        if predicted_escalation and not case["should_escalate"]:
            false_escalations += 1

        # retrieval metrics (ONLY when system did NOT escalate)
        if not predicted_escalation:
            total_retrieval += 1
            if predicted_policy == case["expected_policy"]:
                correct_retrieval += 1

    print("Retrieval Accuracy (non-escalated only):", correct_retrieval / max(1, total_retrieval))
    print("Coverage (non-escalated fraction):", total_retrieval / total)
    print("Escalation Accuracy:", correct_escalation / total)
    print("False Escalation Rate:", false_escalations / total)


In [116]:
def evaluate_multi_policy_system_v2(test_cases):
    interaction_logs.clear()

    correct_retrieval = 0
    total_retrieval = 0

    correct_escalation = 0
    false_escalations = 0

    total = len(test_cases)

    for case in test_cases:
        handle_complaint(case["text"])
        last_log = interaction_logs[-1]

        predicted_policy = last_log["policy_id"]
        predicted_escalation = last_log["escalated"]

        expected_policies = case.get("expected_policies", [])
        expected_escalate = case["should_escalate"]

        if predicted_escalation == expected_escalate:
            correct_escalation += 1
        if predicted_escalation and not expected_escalate:
            false_escalations += 1

        if not predicted_escalation:
            total_retrieval += 1
            if (predicted_policy in expected_policies) or (predicted_policy is None and expected_policies == []):
                correct_retrieval += 1

    print("Multi Retrieval Accuracy (non-escalated only):", correct_retrieval / max(1, total_retrieval))
    print("Multi Coverage (non-escalated fraction):", total_retrieval / total)
    print("Multi Escalation Accuracy:", correct_escalation / total)
    print("Multi False Escalation Rate:", false_escalations / total)


In [117]:
evaluate_system_v2(test_cases)
evaluate_multi_policy_system_v2(multi_policy_test_cases)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kgw2d44ze6erffjmqax1d459` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99944, Requested 150. Please try again in 1m21.216s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [118]:
def evaluate_decision_only(test_cases):
    correct_retrieval = 0
    total_retrieval = 0
    correct_escalation = 0
    false_escalations = 0
    total = len(test_cases)

    for case in test_cases:
        decision = decision_layer(case["text"])

        predicted_escalation = decision["escalate"]
        expected_escalate = case["should_escalate"]

        if predicted_escalation == expected_escalate:
            correct_escalation += 1
        if predicted_escalation and not expected_escalate:
            false_escalations += 1

        if not predicted_escalation:
            total_retrieval += 1
            predicted_policy = decision["policy"]["policy_id"]
            if predicted_policy == case["expected_policy"]:
                correct_retrieval += 1

    print("Retrieval Accuracy (non-escalated):", correct_retrieval / max(1,total_retrieval))
    print("Coverage:", total_retrieval / total)
    print("Escalation Accuracy:", correct_escalation / total)
    print("False Escalation Rate:", false_escalations / total)


In [119]:
def evaluate_decision_only_multi(test_cases):
    correct_retrieval = 0
    total_retrieval = 0
    correct_escalation = 0
    false_escalations = 0
    total = len(test_cases)

    for case in test_cases:
        decision = decision_layer(case["text"])

        predicted_escalation = decision["escalate"]
        expected_escalate = case["should_escalate"]

        if predicted_escalation == expected_escalate:
            correct_escalation += 1
        if predicted_escalation and not expected_escalate:
            false_escalations += 1

        if not predicted_escalation:
            total_retrieval += 1
            predicted_policy = decision["policy"]["policy_id"]
            expected_policies = case.get("expected_policies", [])
            if predicted_policy in expected_policies:
                correct_retrieval += 1

    print("Multi Retrieval Accuracy (non-escalated):", correct_retrieval / max(1,total_retrieval))
    print("Multi Coverage:", total_retrieval / total)
    print("Multi Escalation Accuracy:", correct_escalation / total)
    print("Multi False Escalation Rate:", false_escalations / total)


In [120]:
evaluate_decision_only(test_cases)
evaluate_decision_only_multi(multi_policy_test_cases)


Retrieval Accuracy (non-escalated): 0.8194444444444444
Coverage: 0.5581395348837209
Escalation Accuracy: 0.5155038759689923
False Escalation Rate: 0.20930232558139536
Multi Retrieval Accuracy (non-escalated): 0.8421052631578947
Multi Coverage: 0.59375
Multi Escalation Accuracy: 0.40625
Multi False Escalation Rate: 0.21875
